In [28]:
a = '''
Write the code in python for:

done() Download the xml from https://registers.esma.europa.eu/solr/esma_registers_firds_files/select?q=*&fq=publication_date:%5B2021-01-17T00:00:00Z+TO+2021-01-19T23:59:59Z%5D&wt=xml&indent=true&start=0&rows=100

From the xml, please parse through to the first download link whose file_type is DLTINS and download the zip
Extract the xml from the zip.
Convert the contents of the xml into a CSV with the following header:
FinInstrmGnlAttrbts.Id
FinInstrmGnlAttrbts.FullNm
FinInstrmGnlAttrbts.ClssfctnTp
FinInstrmGnlAttrbts.CmmdtyDerivInd
FinInstrmGnlAttrbts.NtnlCcy
Issr
Store the csv from step in an AWS S3 bucket
The above function should be run as an AWS Lambda (Optional)'''

In [29]:
import requests
import xml.etree.ElementTree as et

res = requests.get("https://registers.esma.europa.eu/solr/esma_registers_firds_files/select?q=*&fq=publication_date:%5B2021-01-17T00:00:00Z+TO+2021-01-19T23:59:59Z%5D&wt=xml&indent=true&start=0&rows=100")
con = res.content
print(res.url)

https://registers.esma.europa.eu/solr/esma_registers_firds_files/select?q=*&fq=publication_date:%5B2021-01-17T00:00:00Z+TO+2021-01-19T23:59:59Z%5D&wt=xml&indent=true&start=0&rows=100


In [30]:
type(con)

bytes

In [31]:
with open('data.xml', 'wb') as f:
    f.write(con)
mytree = et.parse('data.xml')

In [32]:
root = mytree.getroot()
for i in root:
    print(i.tag, "  gahaha    ",i.attrib)
print(root.tag,"\n\n", root.attrib,"\n\n", root.text,"\n\n",root.iter)

lst   gahaha     {'name': 'responseHeader'}
result   gahaha     {'name': 'response', 'numFound': '4', 'start': '0'}
response 

 {} 

 

 

 <built-in method iter of xml.etree.ElementTree.Element object at 0x0000028B802FF6A0>


In [33]:
xml_str = ET.tostring(tree.getroot(), encoding='unicode', method='xml')

# Use minidom to pretty-print the XML string
xml_dom = minidom.parseString(xml_str)
pretty_xml_str = xml_dom.toprettyxml()

# Print the formatted XML string
print(pretty_xml_str)

NameError: name 'minidom' is not defined

In [ ]:
root = mytree.getroot()
print(root[0].tag,"\n\n", root[0].attrib,"\n\n", root[0].text,"\n\n",root[0].iter)

In [ ]:
root = mytree.getroot()
print(root[1].tag,"\n\n", root[1].attrib,"\n\n", root[1].text,"\n\n",root[1].iter)

In [ ]:
root = mytree.getroot()
print(root[0][2][0].tag,"\n\n", root[0][0].attrib,"\n\n", root[0][0].text,"\n\n",root[0][0].iter)

In [ ]:
root = mytree.getroot()
print(root.tag,"\n\n", root.attrib,"\n\n", root.text,"\n\n",root.iter)

In [ ]:
download_links = root.findall(".//str[@name='download_link']")

In [34]:
for i in download_links:
    i = str(i)
    print(i)

NameError: name 'download_links' is not defined

In [35]:
for i in download_links:
    
    if 'DLTINS' in i.text:
        link =i.text
        break

NameError: name 'download_links' is not defined

In [36]:
print(link)

NameError: name 'link' is not defined

In [37]:
response = requests.get(link)

NameError: name 'link' is not defined

In [38]:
if response.status_code ==200:
    with open("DLTINS.zip","wb") as f:
        f.write(response.content)
    print("File Dowloaded")
else:
    print("error aa gya bhai")

NameError: name 'response' is not defined

The wb mode stands for "write binary" mode in Python. It is used for writing data to a binary file. When downloading a binary file, such as a zip file, it is important to open the file in binary mode, as this ensures that the data is read and written correctly, without any modification of bytes.

If you open a binary file in text mode (w), Python may perform certain text encoding and decoding operations which can result in corrupted data. Therefore, it is best practice to always open binary files in binary mode.

In [39]:
import zipfile

In [40]:
with zipfile.ZipFile("DLTINS.zip", 'r') as f:
    p =f.namelist()
    print(len(p))
    print(type(p))
    print(*p)
    for i in p:
        name =i
        f.extract(i)

1
<class 'list'>
DLTINS_20210117_01of01.xml


In [41]:
tree = et.parse('DLTINS_20210117_01of01.xml')
root = tree.getroot()

In [42]:
print(tree)

In [43]:
print(root.tag)


{urn:iso:std:iso:20022:tech:xsd:head.003.001.01}BizData


In [44]:
print(root.tag)
for child in root:
    print(child.tag, child.attrib)

{urn:iso:std:iso:20022:tech:xsd:head.003.001.01}BizData
{urn:iso:std:iso:20022:tech:xsd:head.003.001.01}Hdr {}
{urn:iso:std:iso:20022:tech:xsd:head.003.001.01}Pyld {}


In [45]:
import xml.etree.ElementTree as ET
import csv
tree = ET.parse('DLTINS_20210117_01of01.xml')
root=tree.getroot()
with open('Parvesh.csv', 'w', newline='') as file:
    write=csv.writer(file)
    headers= ['Id','FullNm','ClssfctnTp','CmmdtyDerivInd','NtnlCcy']
    write.writerow(headers)
    for i in root.findall('.//doc'):
        row=[id = doc.find('str[@name="Id"]').text,
            FullNm = doc.find('str[@name="FullNm"]').text,
            ClssfctnTp = doc.find('str[@name="ClssfctnTp"]').text,
            CmmdtyDerivInd = doc.find('str[@name="CmmdtyDerivInd"]').text,
            NtnlCcy = doc.find('str[@name="NtnlCcy"]').text]
        write.writerow(row)
with open('Parvesh.csv', newline='')as csvfile:
    reader= csv.reader(csvfile)
    for row in reader:
        print(','.join(row))

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (1404856470.py, line 10)

In [ ]:
import xml.etree.ElementTree as et
xml_file='DLTINS_20210117_01of01.xml'
with open(xml_file, 'r', encoding='utf-8') as file:
    data=file.read()
    print(data)

In [ ]:
import xml.etree.ElementTree as et
xml_file='DLTINS_20210117_01of01.xml'

In [ ]:
mytree = et.parse(xml_file)
# Print the root element and its children
print(root.tag)
for child in root:
    print(child.tag, child.attrib)

In [ ]:
print(root.text)

In [ ]:
import pandas as pd

# Create an empty list to store the data
data = []

# Loop through each child element of the root element
for child in root.iter():
    # Extract the data from the element
    data_dict = {}
    for attribute in child.attrib:
        data_dict[attribute] = child.attrib[attribute]
    data.append(data_dict)

# Create a pandas DataFrame from the list of dictionaries
df = pd.DataFrame(data)


In [ ]:
df.describe()

In [ ]:
for i in root:
    print(i.)

In [ ]:
import json
data = {}
for child in root:
    data[child.tag] = child.text

# Convert the Python dictionary to JSON and print it
json_data = json.dumps(data)
print(json_data)


In [ ]:
json_data.get("FinInstrmGnlAttrbts")

In [ ]:
import xml.etree.ElementTree as et
import csv

In [ ]:
my_string ='''<?xml version="1.0" encoding="UTF-8"?>
<BizData xsi:schemaLocation="urn:iso:std:iso:20022:tech:xsd:head.003.001.01 head.003.001.01.xsd" xmlns="urn:iso:std:iso:20022:tech:xsd:head.003.001.01" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"><Hdr><AppHdr xsi:schemaLocation="urn:iso:std:iso:20022:tech:xsd:head.001.001.01 head.001.001.01_ESMAUG_1.0.0.xsd" xmlns="urn:iso:std:iso:20022:tech:xsd:head.001.001.01" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"><Fr><OrgId><Id><OrgId><Othr><Id>EU</Id></Othr></OrgId></Id></OrgId></Fr><To><OrgId><Id><OrgId><Othr><Id>Public</Id></Othr></OrgId></Id></OrgId></To><BizMsgIdr>DLTINS_20210117_01of01</BizMsgIdr><MsgDefIdr>auth.036.001.02</MsgDefIdr><CreDt>2021-01-17T01:17:07.027Z</CreDt></AppHdr></Hdr><Pyld><Document xsi:schemaLocation="urn:iso:std:iso:20022:tech:xsd:auth.036.001.02 auth.036.001.02_ESMAUG_DLTINS_1.1.0.xsd" xmlns="urn:iso:std:iso:20022:tech:xsd:auth.036.001.02" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"><FinInstrmRptgRefDataDltaRpt><RptHdr><RptgNtty><NtlCmptntAuthrty>EU</NtlCmptntAuthrty></RptgNtty><RptgPrd><Dt>2021-01-17</Dt></RptgPrd></RptHdr><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A1R07V3</Id><FullNm>Kreditanst.f.Wiederaufbau     Anl.v.2014 (2021)</FullNm><ShrtNm>KFW/1.625 ANL 20210115 GGAR</ShrtNm><ClssfctnTp>DBFTFB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>549300GDPG70E3MBBU98</Issr><TradgVnRltdAttrbts><Id>XEUB</Id><IssrReq>false</IssrReq><FrstTradDt>2014-01-14T09:00:00Z</FrstTradDt><TermntnDt>2017-12-15T00:00:00Z</TermntnDt></TradgVnRltdAttrbts><DebtInstrmAttrbts><TtlIssdNmnlAmt Ccy="EUR">5000000000</TtlIssdNmnlAmt><MtrtyDt>2021-01-15</MtrtyDt><NmnlValPerUnit Ccy="EUR">1000</NmnlValPerUnit><IntrstRate><Fxd>1.625</Fxd></IntrstRate></DebtInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><RlvntTradgVn>MANL</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A1R07V3</Id><FullNm>KFW 1 5/8 01/15/21</FullNm><ShrtNm>KFW/1.625 ANL 20210115 GGAR</ShrtNm><ClssfctnTp>DBFTFB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>549300GDPG70E3MBBU98</Issr><TradgVnRltdAttrbts><Id>XEUM</Id><IssrReq>false</IssrReq><FrstTradDt>2014-01-15T06:00:00Z</FrstTradDt><TermntnDt>2020-12-30T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DebtInstrmAttrbts><TtlIssdNmnlAmt Ccy="EUR">5000000000</TtlIssdNmnlAmt><MtrtyDt>2021-01-15</MtrtyDt><NmnlValPerUnit Ccy="EUR">1000</NmnlValPerUnit><IntrstRate><Fxd>1.625</Fxd></IntrstRate></DebtInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><RlvntTradgVn>MANL</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A1R07V3</Id><FullNm>Kreditanst.f.Wiederaufbau Anl.v.2014 (2021)</FullNm><ShrtNm>KFW/1.625 ANL 20210115 GGAR</ShrtNm><ClssfctnTp>DBFTFB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>549300GDPG70E3MBBU98</Issr><TradgVnRltdAttrbts><Id>XGAT</Id><IssrReq>false</IssrReq><FrstTradDt>2014-01-14T06:00:00Z</FrstTradDt><TermntnDt>2021-01-12T21:00:00Z</TermntnDt></TradgVnRltdAttrbts><DebtInstrmAttrbts><TtlIssdNmnlAmt Ccy="EUR">5000000000</TtlIssdNmnlAmt><MtrtyDt>2021-01-15</MtrtyDt><NmnlValPerUnit Ccy="EUR">1000</NmnlValPerUnit><IntrstRate><Fxd>1.625</Fxd></IntrstRate></DebtInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><RlvntTradgVn>MANL</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A1R07V3</Id><FullNm>Kreditanst.f.Wiederaufbau Anl.v.2014 (2021)</FullNm><ShrtNm>KFW/1.625 ANL 20210115 GGAR</ShrtNm><ClssfctnTp>DBFTFB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>549300GDPG70E3MBBU98</Issr><TradgVnRltdAttrbts><Id>XGRM</Id><IssrReq>false</IssrReq><FrstTradDt>2014-01-14T08:00:00Z</FrstTradDt><TermntnDt>2014-01-14T08:00:00Z</TermntnDt></TradgVnRltdAttrbts><DebtInstrmAttrbts><TtlIssdNmnlAmt Ccy="EUR">5000000000</TtlIssdNmnlAmt><MtrtyDt>2021-01-15</MtrtyDt><NmnlValPerUnit Ccy="EUR">1000</NmnlValPerUnit><IntrstRate><Fxd>1.625</Fxd></IntrstRate></DebtInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><RlvntTradgVn>MANL</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A1X3J56</Id><FullNm>IKB Deutsche Industriebank AG Stufenz.MTN-IHS v.2014(2021)</FullNm><ShrtNm>IKB DT.INDRIEBK/VAR MTI 20210120</ShrtNm><ClssfctnTp>DTVUFB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>PWEFG14QWWESISQ84C69</Issr><TradgVnRltdAttrbts><Id>DUSB</Id><IssrReq>true</IssrReq><AdmssnApprvlDtByIssr>2014-01-13T09:24:00Z</AdmssnApprvlDtByIssr><FrstTradDt>2014-01-20T08:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DebtInstrmAttrbts><TtlIssdNmnlAmt Ccy="EUR">10000000</TtlIssdNmnlAmt><MtrtyDt>2021-01-20</MtrtyDt><NmnlValPerUnit Ccy="EUR">1000</NmnlValPerUnit><IntrstRate><Fxd>4.5</Fxd></IntrstRate><DebtSnrty>SNDB</DebtSnrty></DebtInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><RlvntTradgVn>DUSB</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A1X3J56</Id><FullNm>IKB Deutsche Industriebank AG Stufenz.MTN-IHS v.2014(2021)</FullNm><ShrtNm>IKB DT.INDRIEBK/VAR MTI 20210120</ShrtNm><ClssfctnTp>DTVUFB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>PWEFG14QWWESISQ84C69</Issr><TradgVnRltdAttrbts><Id>FRAB</Id><IssrReq>false</IssrReq><FrstTradDt>2014-01-20T07:00:00Z</FrstTradDt><TermntnDt>2021-01-15T22:59:59Z</TermntnDt></TradgVnRltdAttrbts><DebtInstrmAttrbts><TtlIssdNmnlAmt Ccy="EUR">10000000</TtlIssdNmnlAmt><MtrtyDt>2021-01-20</MtrtyDt><NmnlValPerUnit Ccy="EUR">1000</NmnlValPerUnit><IntrstRate><Fxd>4.5</Fxd></IntrstRate><DebtSnrty>SNDB</DebtSnrty></DebtInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><RlvntTradgVn>DUSB</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A1X3J56</Id><FullNm>LSFEU  3.700  1/20/21 (URegS)</FullNm><ShrtNm>IKB DT.INDRIEBK/VAR MTI 20210120</ShrtNm><ClssfctnTp>DTVUFB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>PWEFG14QWWESISQ84C69</Issr><TradgVnRltdAttrbts><Id>MAEL</Id><IssrReq>false</IssrReq><FrstTradDt>2018-01-03T00:00:00Z</FrstTradDt><TermntnDt>2019-11-15T00:00:00Z</TermntnDt></TradgVnRltdAttrbts><DebtInstrmAttrbts><TtlIssdNmnlAmt Ccy="EUR">10000000</TtlIssdNmnlAmt><MtrtyDt>2021-01-20</MtrtyDt><NmnlValPerUnit Ccy="EUR">1000</NmnlValPerUnit><IntrstRate><Fxd>4.5</Fxd></IntrstRate><DebtSnrty>SNDB</DebtSnrty></DebtInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><RlvntTradgVn>DUSB</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A1YC5L8</Id><FullNm>NIESA Float 01/15/21 BOND</FullNm><ShrtNm>NIEDERSACHSEN/VAR LSA 20210115 GGAR</ShrtNm><ClssfctnTp>DNVTFB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>391200ITQQZ7JMHXK080</Issr><TradgVnRltdAttrbts><Id>AURO</Id><IssrReq>false</IssrReq><FrstTradDt>2020-07-03T20:05:35Z</FrstTradDt><TermntnDt>2021-01-15T00:00:00Z</TermntnDt></TradgVnRltdAttrbts><DebtInstrmAttrbts><TtlIssdNmnlAmt Ccy="EUR">1000000000</TtlIssdNmnlAmt><MtrtyDt>2021-01-15</MtrtyDt><NmnlValPerUnit Ccy="EUR">1000</NmnlValPerUnit><IntrstRate><Fltg><RefRate><ISIN>EU0009652783</ISIN></RefRate><Term><Unit>MNTH</Unit><Val>3</Val></Term><BsisPtSprd>10</BsisPtSprd></Fltg></IntrstRate></DebtInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-15</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>MANL</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A1YC5L8</Id><FullNm>NIESA Float 01/15/21 BOND</FullNm><ShrtNm>NIEDERSACHSEN/VAR LSA 20210115 GGAR</ShrtNm><ClssfctnTp>DNVTFB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>391200ITQQZ7JMHXK080</Issr><TradgVnRltdAttrbts><Id>BTFE</Id><IssrReq>false</IssrReq><FrstTradDt>2020-08-28T08:54:05.797Z</FrstTradDt><TermntnDt>2021-01-15T00:00:00Z</TermntnDt></TradgVnRltdAttrbts><DebtInstrmAttrbts><TtlIssdNmnlAmt Ccy="EUR">1000000000</TtlIssdNmnlAmt><MtrtyDt>2021-01-15</MtrtyDt><NmnlValPerUnit Ccy="EUR">1000</NmnlValPerUnit><IntrstRate><Fltg><RefRate><ISIN>EU0009652783</ISIN></RefRate><Term><Unit>MNTH</Unit><Val>3</Val></Term><BsisPtSprd>10</BsisPtSprd></Fltg></IntrstRate></DebtInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-15</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>MANL</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A1YC5L8</Id><FullNm>NIESA  1/15/21 (FRN) NIEDERSACHSEN</FullNm><ShrtNm>NIEDERSACHSEN/VAR LSA 20210115 GGAR</ShrtNm><ClssfctnTp>DNVTFB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>391200ITQQZ7JMHXK080</Issr><TradgVnRltdAttrbts><Id>MANL</Id><IssrReq>false</IssrReq><FrstTradDt>2019-04-16T12:15:19Z</FrstTradDt><TermntnDt>2021-01-15T22:39:22Z</TermntnDt></TradgVnRltdAttrbts><DebtInstrmAttrbts><TtlIssdNmnlAmt Ccy="EUR">1000000000</TtlIssdNmnlAmt><MtrtyDt>2021-01-15</MtrtyDt><NmnlValPerUnit Ccy="EUR">1000</NmnlValPerUnit><IntrstRate><Fltg><RefRate><ISIN>EU0009652783</ISIN></RefRate><Term><Unit>MNTH</Unit><Val>3</Val></Term><BsisPtSprd>10</BsisPtSprd></Fltg></IntrstRate></DebtInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>MANL</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A2252F1</Id><FullNm>EURJPY Long 125.4520201229</FullNm><ShrtNm>Raydius GMBH/COS CRRATE UNL</ShrtNm><ClssfctnTp>RFCTCB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>213800MYZ42K9PTIJ680</Issr><TradgVnRltdAttrbts><Id>SPEX</Id><IssrReq>true</IssrReq><AdmssnApprvlDtByIssr>2020-12-29T22:01:30Z</AdmssnApprvlDtByIssr><FrstTradDt>2020-12-29T22:01:30Z</FrstTradDt><TermntnDt>2021-01-15T15:04:54Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><PricMltplr>1</PricMltplr><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="EUR">125.6164</Amt><Sgn>true</Sgn></MntryVal></Pric></StrkPric><OptnExrcStyle>BERM</OptnExrcStyle><DlvryTp>CASH</DlvryTp><AsstClssSpcfcAttrbts><FX><FxTp>FXMJ</FxTp><OthrNtnlCcy>JPY</OthrNtnlCcy></FX></AsstClssSpcfcAttrbts></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>SPEX</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A2253D4</Id><FullNm>Italy 40 Long 2230020210106</FullNm><ShrtNm>Raydius GMBH/COS FTSEMI UNL</ShrtNm><ClssfctnTp>RFITCB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>213800MYZ42K9PTIJ680</Issr><TradgVnRltdAttrbts><Id>SPEX</Id><IssrReq>true</IssrReq><AdmssnApprvlDtByIssr>2021-01-06T22:01:30Z</AdmssnApprvlDtByIssr><FrstTradDt>2021-01-06T22:01:30Z</FrstTradDt><TermntnDt>2021-01-15T15:20:47Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><PricMltplr>0.01</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IT0003465736</ISIN><Nm><RefRate><Nm>FTSE MIB</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="EUR">22309.46</Amt><Sgn>true</Sgn></MntryVal></Pric></StrkPric><OptnExrcStyle>BERM</OptnExrcStyle><DlvryTp>CASH</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>SPEX</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A2253N3</Id><FullNm>France 40 Long 557520210112</FullNm><ShrtNm>Raydius GMBH/COS CAC40I UNL</ShrtNm><ClssfctnTp>RFITCB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>213800MYZ42K9PTIJ680</Issr><TradgVnRltdAttrbts><Id>SPEX</Id><IssrReq>true</IssrReq><AdmssnApprvlDtByIssr>2021-01-12T22:01:30Z</AdmssnApprvlDtByIssr><FrstTradDt>2021-01-12T22:01:30Z</FrstTradDt><TermntnDt>2021-01-15T15:20:30Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><PricMltplr>0.01</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>FR0003500008</ISIN><Nm><RefRate><Nm>CAC 40</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="EUR">5575.59</Amt><Sgn>true</Sgn></MntryVal></Pric></StrkPric><OptnExrcStyle>BERM</OptnExrcStyle><DlvryTp>CASH</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>SPEX</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A2253R4</Id><FullNm>Spain 35 Long 825020210112</FullNm><ShrtNm>Raydius GMBH/COS IBEX35 UNL</ShrtNm><ClssfctnTp>RFITCB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>213800MYZ42K9PTIJ680</Issr><TradgVnRltdAttrbts><Id>SPEX</Id><IssrReq>true</IssrReq><AdmssnApprvlDtByIssr>2021-01-12T22:01:30Z</AdmssnApprvlDtByIssr><FrstTradDt>2021-01-12T22:01:30Z</FrstTradDt><TermntnDt>2021-01-15T14:40:10Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><PricMltplr>0.01</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>ES0SI0000005</ISIN><Nm><RefRate><Nm>IBEX 35</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="EUR">8250.88</Amt><Sgn>true</Sgn></MntryVal></Pric></StrkPric><OptnExrcStyle>BERM</OptnExrcStyle><DlvryTp>CASH</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>SPEX</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A225468</Id><FullNm>US Tech 100 Long 12787.520210113</FullNm><ShrtNm>Raydius GMBH/COS NASDAQ UNL</ShrtNm><ClssfctnTp>RFITCB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>213800MYZ42K9PTIJ680</Issr><TradgVnRltdAttrbts><Id>SPEX</Id><IssrReq>true</IssrReq><AdmssnApprvlDtByIssr>2021-01-13T22:01:30Z</AdmssnApprvlDtByIssr><FrstTradDt>2021-01-13T22:01:30Z</FrstTradDt><TermntnDt>2021-01-15T15:20:10Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><PricMltplr>0.01</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>US6311011026</ISIN><Nm><RefRate><Nm>NASDAQ 100</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">12788.42</Amt><Sgn>true</Sgn></MntryVal></Pric></StrkPric><OptnExrcStyle>BERM</OptnExrcStyle><DlvryTp>CASH</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>SPEX</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A225492</Id><FullNm>Italy 40 Long 2230020210114</FullNm><ShrtNm>Raydius GMBH/COS FTSEMI UNL</ShrtNm><ClssfctnTp>RFITCB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>213800MYZ42K9PTIJ680</Issr><TradgVnRltdAttrbts><Id>SPEX</Id><IssrReq>true</IssrReq><AdmssnApprvlDtByIssr>2021-01-14T22:01:30Z</AdmssnApprvlDtByIssr><FrstTradDt>2021-01-14T22:01:30Z</FrstTradDt><TermntnDt>2021-01-15T15:20:57Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><PricMltplr>0.01</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IT0003465736</ISIN><Nm><RefRate><Nm>FTSE MIB</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="EUR">22300</Amt><Sgn>true</Sgn></MntryVal></Pric></StrkPric><OptnExrcStyle>BERM</OptnExrcStyle><DlvryTp>CASH</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>SPEX</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A2254A8</Id><FullNm>Italy 40 Long 2250020210114</FullNm><ShrtNm>Raydius GMBH/COS FTSEMI UNL</ShrtNm><ClssfctnTp>RFITCB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>213800MYZ42K9PTIJ680</Issr><TradgVnRltdAttrbts><Id>SPEX</Id><IssrReq>true</IssrReq><AdmssnApprvlDtByIssr>2021-01-14T22:01:30Z</AdmssnApprvlDtByIssr><FrstTradDt>2021-01-14T22:01:30Z</FrstTradDt><TermntnDt>2021-01-15T14:12:29Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><PricMltplr>0.01</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IT0003465736</ISIN><Nm><RefRate><Nm>FTSE MIB</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="EUR">22500</Amt><Sgn>true</Sgn></MntryVal></Pric></StrkPric><OptnExrcStyle>BERM</OptnExrcStyle><DlvryTp>CASH</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>SPEX</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A2254B6</Id><FullNm>US Tech 100 Long 12862.520210114</FullNm><ShrtNm>Raydius GMBH/COS NASDAQ UNL</ShrtNm><ClssfctnTp>RFITCB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>213800MYZ42K9PTIJ680</Issr><TradgVnRltdAttrbts><Id>SPEX</Id><IssrReq>true</IssrReq><AdmssnApprvlDtByIssr>2021-01-14T22:01:30Z</AdmssnApprvlDtByIssr><FrstTradDt>2021-01-14T22:01:30Z</FrstTradDt><TermntnDt>2021-01-15T15:05:17Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><PricMltplr>0.01</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>US6311011026</ISIN><Nm><RefRate><Nm>NASDAQ 100</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">12862.5</Amt><Sgn>true</Sgn></MntryVal></Pric></StrkPric><OptnExrcStyle>BERM</OptnExrcStyle><DlvryTp>CASH</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>SPEX</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A2254D2</Id><FullNm>FTSE 100 Long 6712.520210114</FullNm><ShrtNm>Raydius GMBH/COS FTS100 UNL</ShrtNm><ClssfctnTp>RFITCB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>213800MYZ42K9PTIJ680</Issr><TradgVnRltdAttrbts><Id>SPEX</Id><IssrReq>true</IssrReq><AdmssnApprvlDtByIssr>2021-01-14T22:01:30Z</AdmssnApprvlDtByIssr><FrstTradDt>2021-01-14T22:01:30Z</FrstTradDt><TermntnDt>2021-01-15T15:05:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><PricMltplr>0.01</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>GB0001383545</ISIN><Nm><RefRate><Nm>FTSE 100</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="GBP">6712.5</Amt><Sgn>true</Sgn></MntryVal></Pric></StrkPric><OptnExrcStyle>BERM</OptnExrcStyle><DlvryTp>CASH</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>SPEX</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A229742</Id><FullNm>EURUSD Long 1.20620201210</FullNm><ShrtNm>Raydius GMBH/COS CRRATE UNL</ShrtNm><ClssfctnTp>RFCTCB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>213800MYZ42K9PTIJ680</Issr><TradgVnRltdAttrbts><Id>SPEX</Id><IssrReq>true</IssrReq><AdmssnApprvlDtByIssr>2020-12-10T04:20:01Z</AdmssnApprvlDtByIssr><FrstTradDt>2020-12-10T04:20:01Z</FrstTradDt><TermntnDt>2021-01-15T14:19:10Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><PricMltplr>100</PricMltplr><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="EUR">1.210594</Amt><Sgn>true</Sgn></MntryVal></Pric></StrkPric><OptnExrcStyle>BERM</OptnExrcStyle><DlvryTp>CASH</DlvryTp><AsstClssSpcfcAttrbts><FX><FxTp>FXMJ</FxTp><OthrNtnlCcy>USD</OthrNtnlCcy></FX></AsstClssSpcfcAttrbts></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>SPEX</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A22NS07</Id><FullNm>Germany 30 Long 1374020210106</FullNm><ShrtNm>Raydius GMBH/COS DAX UNL</ShrtNm><ClssfctnTp>RFITCB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>213800MYZ42K9PTIJ680</Issr><TradgVnRltdAttrbts><Id>SPEX</Id><IssrReq>true</IssrReq><AdmssnApprvlDtByIssr>2021-01-06T17:05:01Z</AdmssnApprvlDtByIssr><FrstTradDt>2021-01-06T17:05:01Z</FrstTradDt><TermntnDt>2021-01-15T15:07:58Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><PricMltplr>0.01</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>DE0008469008</ISIN><Nm><RefRate><Nm>DAX</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="EUR">13746.54</Amt><Sgn>true</Sgn></MntryVal></Pric></StrkPric><OptnExrcStyle>BERM</OptnExrcStyle><DlvryTp>CASH</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>SPEX</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A22NS64</Id><FullNm>Germany 30 Long 1386020210111</FullNm><ShrtNm>Raydius GMBH/COS DAX UNL</ShrtNm><ClssfctnTp>RFITCB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>213800MYZ42K9PTIJ680</Issr><TradgVnRltdAttrbts><Id>SPEX</Id><IssrReq>true</IssrReq><AdmssnApprvlDtByIssr>2021-01-11T16:50:02Z</AdmssnApprvlDtByIssr><FrstTradDt>2021-01-11T16:50:02Z</FrstTradDt><TermntnDt>2021-01-15T12:05:30Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><PricMltplr>0.01</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>DE0008469008</ISIN><Nm><RefRate><Nm>DAX</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="EUR">13862.95</Amt><Sgn>true</Sgn></MntryVal></Pric></StrkPric><OptnExrcStyle>BERM</OptnExrcStyle><DlvryTp>CASH</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>SPEX</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A22PE84</Id><FullNm>Wall Street Long 3062520210106</FullNm><ShrtNm>Raydius GMBH/COS DJIA UNL</ShrtNm><ClssfctnTp>RFITCB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>213800MYZ42K9PTIJ680</Issr><TradgVnRltdAttrbts><Id>SPEX</Id><IssrReq>true</IssrReq><AdmssnApprvlDtByIssr>2021-01-06T16:10:02Z</AdmssnApprvlDtByIssr><FrstTradDt>2021-01-06T16:10:02Z</FrstTradDt><TermntnDt>2021-01-15T15:21:30Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><PricMltplr>0.01</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>US2605661048</ISIN><Nm><RefRate><Nm>Dow Jones Industrial Avg</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">30637.91</Amt><Sgn>true</Sgn></MntryVal></Pric></StrkPric><OptnExrcStyle>BERM</OptnExrcStyle><DlvryTp>CASH</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>SPEX</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A22PF26</Id><FullNm>US 500 Long 376520210107</FullNm><ShrtNm>Raydius GMBH/COS S&amp;P500 UNL</ShrtNm><ClssfctnTp>RFITCB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>213800MYZ42K9PTIJ680</Issr><TradgVnRltdAttrbts><Id>SPEX</Id><IssrReq>true</IssrReq><AdmssnApprvlDtByIssr>2021-01-07T15:05:01Z</AdmssnApprvlDtByIssr><FrstTradDt>2021-01-07T15:05:01Z</FrstTradDt><TermntnDt>2021-01-15T15:07:37Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><PricMltplr>0.01</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>US78378X1072</ISIN><Nm><RefRate><Nm>S&amp;P 500</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">3766.1</Amt><Sgn>true</Sgn></MntryVal></Pric></StrkPric><OptnExrcStyle>BERM</OptnExrcStyle><DlvryTp>CASH</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>SPEX</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A22PF42</Id><FullNm>Wall Street Long 3087520210111</FullNm><ShrtNm>Raydius GMBH/COS DJIA UNL</ShrtNm><ClssfctnTp>RFITCB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>213800MYZ42K9PTIJ680</Issr><TradgVnRltdAttrbts><Id>SPEX</Id><IssrReq>true</IssrReq><AdmssnApprvlDtByIssr>2021-01-11T17:00:03Z</AdmssnApprvlDtByIssr><FrstTradDt>2021-01-11T17:00:03Z</FrstTradDt><TermntnDt>2021-01-15T14:30:02Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><PricMltplr>0.01</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>US2605661048</ISIN><Nm><RefRate><Nm>Dow Jones Industrial Avg</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">30883.89</Amt><Sgn>true</Sgn></MntryVal></Pric></StrkPric><OptnExrcStyle>BERM</OptnExrcStyle><DlvryTp>CASH</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>SPEX</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A22PGV0</Id><FullNm>Wall Street Short 3087520210115</FullNm><ShrtNm>Raydius GMBH/POS DJIA UNL</ShrtNm><ClssfctnTp>RFITPB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>213800MYZ42K9PTIJ680</Issr><TradgVnRltdAttrbts><Id>SPEX</Id><IssrReq>true</IssrReq><AdmssnApprvlDtByIssr>2021-01-15T16:05:01Z</AdmssnApprvlDtByIssr><FrstTradDt>2021-01-15T16:05:01Z</FrstTradDt><TermntnDt>2021-01-15T16:38:08Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><PricMltplr>0.01</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>US2605661048</ISIN><Nm><RefRate><Nm>Dow Jones Industrial Avg</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>PUTO</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">30875</Amt><Sgn>true</Sgn></MntryVal></Pric></StrkPric><OptnExrcStyle>BERM</OptnExrcStyle><DlvryTp>CASH</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>SPEX</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A22PY80</Id><FullNm>Gold Long 183020210111</FullNm><ShrtNm>Raydius GMBH/COS GOLD UNL</ShrtNm><ClssfctnTp>RFTTCB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>true</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>213800MYZ42K9PTIJ680</Issr><TradgVnRltdAttrbts><Id>SPEX</Id><IssrReq>true</IssrReq><AdmssnApprvlDtByIssr>2021-01-11T17:30:01Z</AdmssnApprvlDtByIssr><FrstTradDt>2021-01-11T17:30:01Z</FrstTradDt><TermntnDt>2021-01-15T15:24:55Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><PricMltplr>0.1</PricMltplr><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">1830.53</Amt><Sgn>true</Sgn></MntryVal></Pric></StrkPric><OptnExrcStyle>BERM</OptnExrcStyle><DlvryTp>CASH</DlvryTp><AsstClssSpcfcAttrbts><Cmmdty><Pdct><Metl><Prcs><BasePdct>METL</BasePdct><SubPdct>PRME</SubPdct><AddtlSubPdct>GOLD</AddtlSubPdct></Prcs></Metl></Pdct></Cmmdty></AsstClssSpcfcAttrbts></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>SPEX</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A22Q1Q9</Id><FullNm>Gold Long 184220210115</FullNm><ShrtNm>Raydius GMBH/COS GOLD UNL</ShrtNm><ClssfctnTp>RFTTCB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>true</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>213800MYZ42K9PTIJ680</Issr><TradgVnRltdAttrbts><Id>SPEX</Id><IssrReq>true</IssrReq><AdmssnApprvlDtByIssr>2021-01-15T02:20:01Z</AdmssnApprvlDtByIssr><FrstTradDt>2021-01-15T02:20:01Z</FrstTradDt><TermntnDt>2021-01-15T13:45:10Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><PricMltplr>0.1</PricMltplr><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">1842</Amt><Sgn>true</Sgn></MntryVal></Pric></StrkPric><OptnExrcStyle>BERM</OptnExrcStyle><DlvryTp>CASH</DlvryTp><AsstClssSpcfcAttrbts><Cmmdty><Pdct><Metl><Prcs><BasePdct>METL</BasePdct><SubPdct>PRME</SubPdct><AddtlSubPdct>GOLD</AddtlSubPdct></Prcs></Metl></Pdct></Cmmdty></AsstClssSpcfcAttrbts></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>SPEX</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A22Q1S5</Id><FullNm>Brent Crude Long 54.9520210111</FullNm><ShrtNm>Raydius GMBH/COS BRENTC UNL</ShrtNm><ClssfctnTp>RFMTCB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>true</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>213800MYZ42K9PTIJ680</Issr><TradgVnRltdAttrbts><Id>SPEX</Id><IssrReq>true</IssrReq><AdmssnApprvlDtByIssr>2021-01-11T18:20:01Z</AdmssnApprvlDtByIssr><FrstTradDt>2021-01-11T18:20:01Z</FrstTradDt><TermntnDt>2021-01-15T15:10:08Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><PricMltplr>1</PricMltplr><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">54.9652</Amt><Sgn>true</Sgn></MntryVal></Pric></StrkPric><OptnExrcStyle>BERM</OptnExrcStyle><DlvryTp>CASH</DlvryTp><AsstClssSpcfcAttrbts><Cmmdty><Pdct><Nrgy><Oil><BasePdct>NRGY</BasePdct><SubPdct>OILP</SubPdct></Oil></Nrgy></Pdct></Cmmdty></AsstClssSpcfcAttrbts></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>SPEX</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A22Q1T3</Id><FullNm>US Crude Long 51.920210112</FullNm><ShrtNm>Raydius GMBH/COS CMECLF UNL</ShrtNm><ClssfctnTp>RFMTCB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>true</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>213800MYZ42K9PTIJ680</Issr><TradgVnRltdAttrbts><Id>SPEX</Id><IssrReq>true</IssrReq><AdmssnApprvlDtByIssr>2021-01-12T07:25:00Z</AdmssnApprvlDtByIssr><FrstTradDt>2021-01-12T07:25:00Z</FrstTradDt><TermntnDt>2021-01-15T15:31:12Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><PricMltplr>1</PricMltplr><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">51.9108</Amt><Sgn>true</Sgn></MntryVal></Pric></StrkPric><OptnExrcStyle>BERM</OptnExrcStyle><DlvryTp>CASH</DlvryTp><AsstClssSpcfcAttrbts><Cmmdty><Pdct><Nrgy><Oil><BasePdct>NRGY</BasePdct><SubPdct>OILP</SubPdct></Oil></Nrgy></Pdct></Cmmdty></AsstClssSpcfcAttrbts></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>SPEX</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A22Q221</Id><FullNm>Gold Short 185420210114</FullNm><ShrtNm>Raydius GMBH/POS GOLD UNL</ShrtNm><ClssfctnTp>RFTTPB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>true</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>213800MYZ42K9PTIJ680</Issr><TradgVnRltdAttrbts><Id>SPEX</Id><IssrReq>true</IssrReq><AdmssnApprvlDtByIssr>2021-01-14T20:20:01Z</AdmssnApprvlDtByIssr><FrstTradDt>2021-01-14T20:20:01Z</FrstTradDt><TermntnDt>2021-01-15T01:00:29Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><PricMltplr>0.1</PricMltplr><OptnTp>PUTO</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">1853.88</Amt><Sgn>true</Sgn></MntryVal></Pric></StrkPric><OptnExrcStyle>BERM</OptnExrcStyle><DlvryTp>CASH</DlvryTp><AsstClssSpcfcAttrbts><Cmmdty><Pdct><Metl><Prcs><BasePdct>METL</BasePdct><SubPdct>PRME</SubPdct><AddtlSubPdct>GOLD</AddtlSubPdct></Prcs></Metl></Pdct></Cmmdty></AsstClssSpcfcAttrbts></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>SPEX</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A22Q338</Id><FullNm>US Crude Long 52.520210114</FullNm><ShrtNm>Raydius GMBH/COS CMECLF UNL</ShrtNm><ClssfctnTp>RFMTCB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>true</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>213800MYZ42K9PTIJ680</Issr><TradgVnRltdAttrbts><Id>SPEX</Id><IssrReq>true</IssrReq><AdmssnApprvlDtByIssr>2021-01-14T18:20:01Z</AdmssnApprvlDtByIssr><FrstTradDt>2021-01-14T18:20:01Z</FrstTradDt><TermntnDt>2021-01-15T15:07:10Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><PricMltplr>1</PricMltplr><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">52.5036</Amt><Sgn>true</Sgn></MntryVal></Pric></StrkPric><OptnExrcStyle>BERM</OptnExrcStyle><DlvryTp>CASH</DlvryTp><AsstClssSpcfcAttrbts><Cmmdty><Pdct><Nrgy><Oil><BasePdct>NRGY</BasePdct><SubPdct>OILP</SubPdct></Oil></Nrgy></Pdct></Cmmdty></AsstClssSpcfcAttrbts></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>SPEX</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A22Q403</Id><FullNm>Brent Crude Long 55.6520210114</FullNm><ShrtNm>Raydius GMBH/COS BRENTC UNL</ShrtNm><ClssfctnTp>RFMTCB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>true</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>213800MYZ42K9PTIJ680</Issr><TradgVnRltdAttrbts><Id>SPEX</Id><IssrReq>true</IssrReq><AdmssnApprvlDtByIssr>2021-01-14T20:20:02Z</AdmssnApprvlDtByIssr><FrstTradDt>2021-01-14T20:20:02Z</FrstTradDt><TermntnDt>2021-01-15T05:54:00Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><PricMltplr>1</PricMltplr><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">55.6538</Amt><Sgn>true</Sgn></MntryVal></Pric></StrkPric><OptnExrcStyle>BERM</OptnExrcStyle><DlvryTp>CASH</DlvryTp><AsstClssSpcfcAttrbts><Cmmdty><Pdct><Nrgy><Oil><BasePdct>NRGY</BasePdct><SubPdct>OILP</SubPdct></Oil></Nrgy></Pdct></Cmmdty></AsstClssSpcfcAttrbts></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>SPEX</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A22Q4B5</Id><FullNm>US Crude Long 53.120210114</FullNm><ShrtNm>Raydius GMBH/COS CMECLF UNL</ShrtNm><ClssfctnTp>RFMTCB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>true</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>213800MYZ42K9PTIJ680</Issr><TradgVnRltdAttrbts><Id>SPEX</Id><IssrReq>true</IssrReq><AdmssnApprvlDtByIssr>2021-01-14T22:35:02Z</AdmssnApprvlDtByIssr><FrstTradDt>2021-01-14T22:35:02Z</FrstTradDt><TermntnDt>2021-01-15T05:48:43Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><PricMltplr>1</PricMltplr><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">53.1</Amt><Sgn>true</Sgn></MntryVal></Pric></StrkPric><OptnExrcStyle>BERM</OptnExrcStyle><DlvryTp>CASH</DlvryTp><AsstClssSpcfcAttrbts><Cmmdty><Pdct><Nrgy><Oil><BasePdct>NRGY</BasePdct><SubPdct>OILP</SubPdct></Oil></Nrgy></Pdct></Cmmdty></AsstClssSpcfcAttrbts></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>SPEX</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A2AAJ76</Id><FullNm>Niedersachsen, Land           Landessch.v.16(21) Ausg.858</FullNm><ShrtNm>NIEDERSACHSEN/0.1 LSA 20210120</ShrtNm><ClssfctnTp>DNFTFB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>391200ITQQZ7JMHXK080</Issr><TradgVnRltdAttrbts><Id>BERB</Id><IssrReq>false</IssrReq><FrstTradDt>2016-01-14T07:00:00Z</FrstTradDt><TermntnDt>2021-01-15T16:30:00Z</TermntnDt></TradgVnRltdAttrbts><DebtInstrmAttrbts><TtlIssdNmnlAmt Ccy="EUR">50000000</TtlIssdNmnlAmt><MtrtyDt>2021-01-20</MtrtyDt><NmnlValPerUnit Ccy="EUR">1000</NmnlValPerUnit><IntrstRate><Fxd>0.1</Fxd></IntrstRate><DebtSnrty>SNDB</DebtSnrty></DebtInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><RlvntTradgVn>HANA</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A2AAJ76</Id><FullNm>Niedersachsen, Land Landessch.v.16(21) Ausg.858</FullNm><ShrtNm>NIEDERSACHSEN/0.1 LSA 20210120</ShrtNm><ClssfctnTp>DNFTFB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>391200ITQQZ7JMHXK080</Issr><TradgVnRltdAttrbts><Id>FRAB</Id><IssrReq>false</IssrReq><FrstTradDt>2016-01-13T08:00:00Z</FrstTradDt><TermntnDt>2021-01-15T22:59:59Z</TermntnDt></TradgVnRltdAttrbts><DebtInstrmAttrbts><TtlIssdNmnlAmt Ccy="EUR">50000000</TtlIssdNmnlAmt><MtrtyDt>2021-01-20</MtrtyDt><NmnlValPerUnit Ccy="EUR">1000</NmnlValPerUnit><IntrstRate><Fxd>0.1</Fxd></IntrstRate><DebtSnrty>SNDB</DebtSnrty></DebtInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><RlvntTradgVn>HANA</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A2AAJ76</Id><FullNm>Niedersachsen, Land Landessch.v.16(21) Ausg.858</FullNm><ShrtNm>NIEDERSACHSEN/0.1 LSA 20210120</ShrtNm><ClssfctnTp>DNFTFB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>391200ITQQZ7JMHXK080</Issr><TradgVnRltdAttrbts><Id>HANA</Id><IssrReq>true</IssrReq><AdmssnApprvlDtByIssr>2016-01-12T09:55:00Z</AdmssnApprvlDtByIssr><ReqForAdmssnDt>2016-01-12T09:55:00Z</ReqForAdmssnDt><FrstTradDt>2016-01-20T08:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DebtInstrmAttrbts><TtlIssdNmnlAmt Ccy="EUR">50000000</TtlIssdNmnlAmt><MtrtyDt>2021-01-20</MtrtyDt><NmnlValPerUnit Ccy="EUR">1000</NmnlValPerUnit><IntrstRate><Fxd>0.1</Fxd></IntrstRate><DebtSnrty>SNDB</DebtSnrty></DebtInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><RlvntTradgVn>HANA</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A2AAJ76</Id><FullNm>SACHAN  0.100  1/20/21</FullNm><ShrtNm>NIEDERSACHSEN/0.1 LSA 20210120</ShrtNm><ClssfctnTp>DNFTFB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>391200ITQQZ7JMHXK080</Issr><TradgVnRltdAttrbts><Id>MAEL</Id><IssrReq>false</IssrReq><FrstTradDt>2018-01-03T00:00:00Z</FrstTradDt><TermntnDt>2019-11-15T00:00:00Z</TermntnDt></TradgVnRltdAttrbts><DebtInstrmAttrbts><TtlIssdNmnlAmt Ccy="EUR">50000000</TtlIssdNmnlAmt><MtrtyDt>2021-01-20</MtrtyDt><NmnlValPerUnit Ccy="EUR">1000</NmnlValPerUnit><IntrstRate><Fxd>0.1</Fxd></IntrstRate><DebtSnrty>SNDB</DebtSnrty></DebtInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><RlvntTradgVn>HANA</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A2AAJ76</Id><FullNm>Niedersachsen 21</FullNm><ShrtNm>NIEDERSACHSEN/0.1 LSA 20210120</ShrtNm><ClssfctnTp>DNFTFB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>391200ITQQZ7JMHXK080</Issr><TradgVnRltdAttrbts><Id>STUB</Id><IssrReq>false</IssrReq><FrstTradDt>2016-01-14T08:00:00Z</FrstTradDt><TermntnDt>2021-01-15T18:00:00Z</TermntnDt></TradgVnRltdAttrbts><DebtInstrmAttrbts><TtlIssdNmnlAmt Ccy="EUR">50000000</TtlIssdNmnlAmt><MtrtyDt>2021-01-20</MtrtyDt><NmnlValPerUnit Ccy="EUR">1000</NmnlValPerUnit><IntrstRate><Fxd>0.1</Fxd></IntrstRate><DebtSnrty>SNDB</DebtSnrty></DebtInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><RlvntTradgVn>HANA</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A2AAJ76</Id><FullNm>NIESA 0.1 01/20/21</FullNm><ShrtNm>NIEDERSACHSEN/0.1 LSA 20210120</ShrtNm><ClssfctnTp>DNFTFB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>391200ITQQZ7JMHXK080</Issr><TradgVnRltdAttrbts><Id>XERE</Id><IssrReq>false</IssrReq><FrstTradDt>2020-05-05T06:00:00Z</FrstTradDt><TermntnDt>2021-01-13T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DebtInstrmAttrbts><TtlIssdNmnlAmt Ccy="EUR">50000000</TtlIssdNmnlAmt><MtrtyDt>2021-01-20</MtrtyDt><NmnlValPerUnit Ccy="EUR">1000</NmnlValPerUnit><IntrstRate><Fxd>0.1</Fxd></IntrstRate><DebtSnrty>SNDB</DebtSnrty></DebtInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><RlvntTradgVn>HANA</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A2AAJ76</Id><FullNm>NIESA 0.1 01/20/21</FullNm><ShrtNm>NIEDERSACHSEN/0.1 LSA 20210120</ShrtNm><ClssfctnTp>DNFTFB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>391200ITQQZ7JMHXK080</Issr><TradgVnRltdAttrbts><Id>XEUM</Id><IssrReq>false</IssrReq><FrstTradDt>2016-01-27T06:00:00Z</FrstTradDt><TermntnDt>2021-01-05T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DebtInstrmAttrbts><TtlIssdNmnlAmt Ccy="EUR">50000000</TtlIssdNmnlAmt><MtrtyDt>2021-01-20</MtrtyDt><NmnlValPerUnit Ccy="EUR">1000</NmnlValPerUnit><IntrstRate><Fxd>0.1</Fxd></IntrstRate><DebtSnrty>SNDB</DebtSnrty></DebtInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><RlvntTradgVn>HANA</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A2BN9W0</Id><FullNm>IKB Deutsche Industriebank AG Festzins Plus 01/21 v.17(21)</FullNm><ShrtNm>IKB DT.INDRIEBK/VAR FLM 20210120</ShrtNm><ClssfctnTp>DTVUFB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>PWEFG14QWWESISQ84C69</Issr><TradgVnRltdAttrbts><Id>DUSB</Id><IssrReq>true</IssrReq><AdmssnApprvlDtByIssr>2017-01-11T11:18:00Z</AdmssnApprvlDtByIssr><FrstTradDt>2017-01-20T08:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DebtInstrmAttrbts><TtlIssdNmnlAmt Ccy="EUR">5000000</TtlIssdNmnlAmt><MtrtyDt>2021-01-20</MtrtyDt><NmnlValPerUnit Ccy="EUR">1000</NmnlValPerUnit><IntrstRate><Fltg><RefRate><ISIN>EU0009652809</ISIN></RefRate><Term><Unit>MNTH</Unit><Val>12</Val></Term><BsisPtSprd>0</BsisPtSprd></Fltg></IntrstRate><DebtSnrty>SNDB</DebtSnrty></DebtInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><RlvntTradgVn>DUSB</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A2BN9W0</Id><FullNm>IKB Deutsche Industriebank AG Festzins Plus 01/21 v.17(21)</FullNm><ShrtNm>IKB DT.INDRIEBK/VAR FLM 20210120</ShrtNm><ClssfctnTp>DTVUFB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>PWEFG14QWWESISQ84C69</Issr><TradgVnRltdAttrbts><Id>FRAB</Id><IssrReq>false</IssrReq><FrstTradDt>2017-01-20T07:00:00Z</FrstTradDt><TermntnDt>2021-01-15T22:59:59Z</TermntnDt></TradgVnRltdAttrbts><DebtInstrmAttrbts><TtlIssdNmnlAmt Ccy="EUR">5000000</TtlIssdNmnlAmt><MtrtyDt>2021-01-20</MtrtyDt><NmnlValPerUnit Ccy="EUR">1000</NmnlValPerUnit><IntrstRate><Fltg><RefRate><ISIN>EU0009652809</ISIN></RefRate><Term><Unit>MNTH</Unit><Val>12</Val></Term><BsisPtSprd>0</BsisPtSprd></Fltg></IntrstRate><DebtSnrty>SNDB</DebtSnrty></DebtInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><RlvntTradgVn>DUSB</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A2BN9W0</Id><FullNm>LSFEU  1.300  1/20/21 (URegS)</FullNm><ShrtNm>IKB DT.INDRIEBK/VAR FLM 20210120</ShrtNm><ClssfctnTp>DTVUFB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>PWEFG14QWWESISQ84C69</Issr><TradgVnRltdAttrbts><Id>MAEL</Id><IssrReq>false</IssrReq><FrstTradDt>2018-01-03T00:00:00Z</FrstTradDt><TermntnDt>2019-11-15T00:00:00Z</TermntnDt></TradgVnRltdAttrbts><DebtInstrmAttrbts><TtlIssdNmnlAmt Ccy="EUR">5000000</TtlIssdNmnlAmt><MtrtyDt>2021-01-20</MtrtyDt><NmnlValPerUnit Ccy="EUR">1000</NmnlValPerUnit><IntrstRate><Fltg><RefRate><ISIN>EU0009652809</ISIN></RefRate><Term><Unit>MNTH</Unit><Val>12</Val></Term><BsisPtSprd>0</BsisPtSprd></Fltg></IntrstRate><DebtSnrty>SNDB</DebtSnrty></DebtInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><RlvntTradgVn>DUSB</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A2E4D18</Id><FullNm>SACHAN 0 01/22/21</FullNm><ShrtNm>SACHSEN-ANH/ MTN 20210122 MTP GGAR</ShrtNm><ClssfctnTp>DYFTXB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900LA98JTSA1MF831</Issr><TradgVnRltdAttrbts><Id>XERE</Id><IssrReq>false</IssrReq><FrstTradDt>2020-01-22T06:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DebtInstrmAttrbts><TtlIssdNmnlAmt Ccy="EUR">100000000</TtlIssdNmnlAmt><MtrtyDt>2021-01-22</MtrtyDt><NmnlValPerUnit Ccy="EUR">100000</NmnlValPerUnit><IntrstRate><Fxd>0</Fxd></IntrstRate><DebtSnrty>SNDB</DebtSnrty></DebtInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>FRAA</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A2E4QE8</Id><FullNm>IKB Deutsche Industriebank AG MTN-IHS v.2017(2021)</FullNm><ShrtNm>IKB DT.INDRIEBK/1.2 MTI 20210120</ShrtNm><ClssfctnTp>DTFUFB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>PWEFG14QWWESISQ84C69</Issr><TradgVnRltdAttrbts><Id>DUSB</Id><IssrReq>true</IssrReq><AdmssnApprvlDtByIssr>2017-07-13T13:43:00Z</AdmssnApprvlDtByIssr><FrstTradDt>2017-07-20T08:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DebtInstrmAttrbts><TtlIssdNmnlAmt Ccy="EUR">10000000</TtlIssdNmnlAmt><MtrtyDt>2021-01-20</MtrtyDt><NmnlValPerUnit Ccy="EUR">1000</NmnlValPerUnit><IntrstRate><Fxd>1.2</Fxd></IntrstRate><DebtSnrty>SNDB</DebtSnrty></DebtInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><RlvntTradgVn>DUSB</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A2E4QE8</Id><FullNm>IKB Deutsche Industriebank AG MTN-IHS v.2017(2021)</FullNm><ShrtNm>IKB DT.INDRIEBK/1.2 MTI 20210120</ShrtNm><ClssfctnTp>DTFUFB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>PWEFG14QWWESISQ84C69</Issr><TradgVnRltdAttrbts><Id>FRAB</Id><IssrReq>false</IssrReq><FrstTradDt>2017-07-20T06:00:00Z</FrstTradDt><TermntnDt>2021-01-15T22:59:59Z</TermntnDt></TradgVnRltdAttrbts><DebtInstrmAttrbts><TtlIssdNmnlAmt Ccy="EUR">10000000</TtlIssdNmnlAmt><MtrtyDt>2021-01-20</MtrtyDt><NmnlValPerUnit Ccy="EUR">1000</NmnlValPerUnit><IntrstRate><Fxd>1.2</Fxd></IntrstRate><DebtSnrty>SNDB</DebtSnrty></DebtInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><RlvntTradgVn>DUSB</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A2E4QE8</Id><FullNm>LSFEU  1.200  1/20/21 (URegS)</FullNm><ShrtNm>IKB DT.INDRIEBK/1.2 MTI 20210120</ShrtNm><ClssfctnTp>DTFUFB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>PWEFG14QWWESISQ84C69</Issr><TradgVnRltdAttrbts><Id>MAEL</Id><IssrReq>false</IssrReq><FrstTradDt>2018-01-03T00:00:00Z</FrstTradDt><TermntnDt>2019-11-15T00:00:00Z</TermntnDt></TradgVnRltdAttrbts><DebtInstrmAttrbts><TtlIssdNmnlAmt Ccy="EUR">10000000</TtlIssdNmnlAmt><MtrtyDt>2021-01-20</MtrtyDt><NmnlValPerUnit Ccy="EUR">1000</NmnlValPerUnit><IntrstRate><Fxd>1.2</Fxd></IntrstRate><DebtSnrty>SNDB</DebtSnrty></DebtInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><RlvntTradgVn>DUSB</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000A3H21V1</Id><FullNm>Velero Immobilien AGNamens-Aktien o.N.</FullNm><ShrtNm>VELERO IMMOBIL/AKT O.N.</ShrtNm><ClssfctnTp>ESVUFR</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900V7S2RCPU0XFA24</Issr><TradgVnRltdAttrbts><Id>BAAD</Id><IssrReq>false</IssrReq><FrstTradDt>2020-12-28T08:00:00Z</FrstTradDt><TermntnDt>2021-01-15T08:10:00Z</TermntnDt></TradgVnRltdAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XETU</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000BHY1257</Id><FullNm>BHH 2 01/22/21</FullNm><ShrtNm>BERLIN HYP AG/2 IHS 20210122 E. 67</ShrtNm><ClssfctnTp>DBFUGB</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900C4RSSBWXBSY931</Issr><TradgVnRltdAttrbts><Id>XERE</Id><IssrReq>false</IssrReq><FrstTradDt>2017-01-06T06:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DebtInstrmAttrbts><TtlIssdNmnlAmt Ccy="EUR">40000000</TtlIssdNmnlAmt><MtrtyDt>2021-01-22</MtrtyDt><NmnlValPerUnit Ccy="EUR">100000</NmnlValPerUnit><IntrstRate><Fxd>2</Fxd></IntrstRate><DebtSnrty>SNDB</DebtSnrty></DebtInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>BERA</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C20QLG9</Id><FullNm>OEMB SI 20210115 PS AM C 94.00 0</FullNm><ShrtNm>Eurex/O 20210115 C AM 94.00</ShrtNm><ClssfctnTp>OCAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-11-21T06:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">94</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C20QLH7</Id><FullNm>OEMB SI 20210115 PS AM P 94.00 0</FullNm><ShrtNm>Eurex/O 20210115 P AM 94.00</ShrtNm><ClssfctnTp>OPAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-11-21T06:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>PUTO</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">94</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C22PH63</Id><FullNm>OQDE SI 20210115 PS AM C 116.00 0</FullNm><ShrtNm>Eurex/O 20210115 C AM 116.00</ShrtNm><ClssfctnTp>OCAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-12-12T00:50:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>100</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE0032895942</ISIN><Nm><RefRate><Nm>ISHARES USD CORP BOND ET</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">116</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-13</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C22PH71</Id><FullNm>OQDE SI 20210115 PS AM P 116.00 0</FullNm><ShrtNm>Eurex/O 20210115 P AM 116.00</ShrtNm><ClssfctnTp>OPAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-12-12T00:50:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>100</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE0032895942</ISIN><Nm><RefRate><Nm>ISHARES USD CORP BOND ET</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>PUTO</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">116</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-13</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C23LB39</Id><FullNm>OHYU SI 20210115 PS AM C 89.00 0</FullNm><ShrtNm>Eurex/O 20210115 C AM 89.00</ShrtNm><ClssfctnTp>OCAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-12-21T00:50:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>100</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B4PY7Y77</ISIN><Nm><RefRate><Nm>ISHARES USD HY CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">89</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-13</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C23LB47</Id><FullNm>OHYU SI 20210115 PS AM P 89.00 0</FullNm><ShrtNm>Eurex/O 20210115 P AM 89.00</ShrtNm><ClssfctnTp>OPAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-12-21T00:50:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>100</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B4PY7Y77</ISIN><Nm><RefRate><Nm>ISHARES USD HY CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>PUTO</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">89</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-13</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C23LB54</Id><FullNm>OHYU SI 20210115 PS AM C 90.00 0</FullNm><ShrtNm>Eurex/O 20210115 C AM 90.00</ShrtNm><ClssfctnTp>OCAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-12-21T00:50:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>100</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B4PY7Y77</ISIN><Nm><RefRate><Nm>ISHARES USD HY CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">90</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-13</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C23LB62</Id><FullNm>OHYU SI 20210115 PS AM P 90.00 0</FullNm><ShrtNm>Eurex/O 20210115 P AM 90.00</ShrtNm><ClssfctnTp>OPAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-12-21T00:50:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>100</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B4PY7Y77</ISIN><Nm><RefRate><Nm>ISHARES USD HY CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>PUTO</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">90</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-13</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C257SG3</Id><FullNm>OEMB SI 20210115 PS AM C 112.00 0</FullNm><ShrtNm>Eurex/O 20210115 C AM 112.00</ShrtNm><ClssfctnTp>OCAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2019-01-07T00:50:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>100</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">112</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-13</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C257SH1</Id><FullNm>OEMB SI 20210115 PS AM P 112.00 0</FullNm><ShrtNm>Eurex/O 20210115 P AM 112.00</ShrtNm><ClssfctnTp>OPAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2019-01-07T00:50:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>100</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>PUTO</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">112</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-13</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C257T68</Id><FullNm>OHYU SI 20210115 PS AM C 105.00 0</FullNm><ShrtNm>Eurex/O 20210115 C AM 105.00</ShrtNm><ClssfctnTp>OCAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2019-01-07T00:50:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>100</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B4PY7Y77</ISIN><Nm><RefRate><Nm>ISHARES USD HY CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">105</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-13</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C257T76</Id><FullNm>OHYU SI 20210115 PS AM P 105.00 0</FullNm><ShrtNm>Eurex/O 20210115 P AM 105.00</ShrtNm><ClssfctnTp>OPAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2019-01-07T00:50:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>100</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B4PY7Y77</ISIN><Nm><RefRate><Nm>ISHARES USD HY CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>PUTO</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">105</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-13</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C25AS31</Id><FullNm>OHYU SI 20210115 PS AM C 106.00 0</FullNm><ShrtNm>Eurex/O 20210115 C AM 106.00</ShrtNm><ClssfctnTp>OCAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2019-01-08T00:50:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B4PY7Y77</ISIN><Nm><RefRate><Nm>ISHARES USD HY CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">106</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C25AS49</Id><FullNm>OHYU SI 20210115 PS AM P 106.00 0</FullNm><ShrtNm>Eurex/O 20210115 P AM 106.00</ShrtNm><ClssfctnTp>OPAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2019-01-08T00:50:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B4PY7Y77</ISIN><Nm><RefRate><Nm>ISHARES USD HY CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>PUTO</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">106</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C25HCD3</Id><FullNm>OEMB SI 20210115 PS AM C 113.00 0</FullNm><ShrtNm>Eurex/O 20210115 C AM 113.00</ShrtNm><ClssfctnTp>OCAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2019-01-10T00:50:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">113</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C25HCE1</Id><FullNm>OEMB SI 20210115 PS AM P 113.00 0</FullNm><ShrtNm>Eurex/O 20210115 P AM 113.00</ShrtNm><ClssfctnTp>OPAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2019-01-10T00:50:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>PUTO</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">113</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C25HD56</Id><FullNm>OHYU SI 20210115 PS AM C 107.00 0</FullNm><ShrtNm>Eurex/O 20210115 C AM 107.00</ShrtNm><ClssfctnTp>OCAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2019-01-10T00:50:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B4PY7Y77</ISIN><Nm><RefRate><Nm>ISHARES USD HY CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">107</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C25HD64</Id><FullNm>OHYU SI 20210115 PS AM P 107.00 0</FullNm><ShrtNm>Eurex/O 20210115 P AM 107.00</ShrtNm><ClssfctnTp>OPAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2019-01-10T00:50:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B4PY7Y77</ISIN><Nm><RefRate><Nm>ISHARES USD HY CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>PUTO</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">107</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C26JNV6</Id><FullNm>OHYU SI 20210115 PS AM C 108.00 0</FullNm><ShrtNm>Eurex/O 20210115 C AM 108.00</ShrtNm><ClssfctnTp>OCAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2019-01-21T00:50:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B4PY7Y77</ISIN><Nm><RefRate><Nm>ISHARES USD HY CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">108</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C26JNW4</Id><FullNm>OHYU SI 20210115 PS AM P 108.00 0</FullNm><ShrtNm>Eurex/O 20210115 P AM 108.00</ShrtNm><ClssfctnTp>OPAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2019-01-21T00:50:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B4PY7Y77</ISIN><Nm><RefRate><Nm>ISHARES USD HY CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>PUTO</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">108</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C26JTJ8</Id><FullNm>OQDE SI 20210115 PS AM C 117.00 0</FullNm><ShrtNm>Eurex/O 20210115 C AM 117.00</ShrtNm><ClssfctnTp>OCAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2019-01-21T00:50:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>100</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE0032895942</ISIN><Nm><RefRate><Nm>ISHARES USD CORP BOND ET</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">117</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-13</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C26JTK6</Id><FullNm>OQDE SI 20210115 PS AM P 117.00 0</FullNm><ShrtNm>Eurex/O 20210115 P AM 117.00</ShrtNm><ClssfctnTp>OPAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2019-01-21T00:50:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>100</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE0032895942</ISIN><Nm><RefRate><Nm>ISHARES USD CORP BOND ET</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>PUTO</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">117</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-13</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C26ZJV0</Id><FullNm>OEMB SI 20210115 PS AM C 114.00 0</FullNm><ShrtNm>Eurex/O 20210115 C AM 114.00</ShrtNm><ClssfctnTp>OCAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2019-01-24T00:50:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">114</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C26ZJW8</Id><FullNm>OEMB SI 20210115 PS AM P 114.00 0</FullNm><ShrtNm>Eurex/O 20210115 P AM 114.00</ShrtNm><ClssfctnTp>OPAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2019-01-24T00:50:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>PUTO</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">114</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C27L1Y1</Id><FullNm>OQDE SI 20210115 PS AM C 118.00 0</FullNm><ShrtNm>Eurex/O 20210115 C AM 118.00</ShrtNm><ClssfctnTp>OCAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2019-01-29T00:50:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>100</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE0032895942</ISIN><Nm><RefRate><Nm>ISHARES USD CORP BOND ET</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">118</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-13</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C27L1Z8</Id><FullNm>OQDE SI 20210115 PS AM P 118.00 0</FullNm><ShrtNm>Eurex/O 20210115 P AM 118.00</ShrtNm><ClssfctnTp>OPAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2019-01-29T00:50:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>100</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE0032895942</ISIN><Nm><RefRate><Nm>ISHARES USD CORP BOND ET</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>PUTO</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">118</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-13</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C27XT80</Id><FullNm>OEMB SI 20210115 PS AM C 115.00 0</FullNm><ShrtNm>Eurex/O 20210115 C AM 115.00</ShrtNm><ClssfctnTp>OCAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2019-02-01T00:50:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">115</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C27XT98</Id><FullNm>OEMB SI 20210115 PS AM P 115.00 0</FullNm><ShrtNm>Eurex/O 20210115 P AM 115.00</ShrtNm><ClssfctnTp>OPAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2019-02-01T00:50:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>PUTO</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">115</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C27XU20</Id><FullNm>OHYU SI 20210115 PS AM C 109.00 0</FullNm><ShrtNm>Eurex/O 20210115 C AM 109.00</ShrtNm><ClssfctnTp>OCAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2019-02-01T00:50:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B4PY7Y77</ISIN><Nm><RefRate><Nm>ISHARES USD HY CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">109</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C27XU38</Id><FullNm>OHYU SI 20210115 PS AM P 109.00 0</FullNm><ShrtNm>Eurex/O 20210115 P AM 109.00</ShrtNm><ClssfctnTp>OPAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2019-02-01T00:50:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B4PY7Y77</ISIN><Nm><RefRate><Nm>ISHARES USD HY CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>PUTO</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">109</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C27XXA8</Id><FullNm>OQDE SI 20210115 PS AM C 119.00 0</FullNm><ShrtNm>Eurex/O 20210115 C AM 119.00</ShrtNm><ClssfctnTp>OCAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2019-02-01T00:50:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>100</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE0032895942</ISIN><Nm><RefRate><Nm>ISHARES USD CORP BOND ET</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">119</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-13</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C27XXB6</Id><FullNm>OQDE SI 20210115 PS AM P 119.00 0</FullNm><ShrtNm>Eurex/O 20210115 P AM 119.00</ShrtNm><ClssfctnTp>OPAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2019-02-01T00:50:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>100</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE0032895942</ISIN><Nm><RefRate><Nm>ISHARES USD CORP BOND ET</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>PUTO</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">119</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-13</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C2VEA80</Id><FullNm>GLJM SI 20210115 CS</FullNm><ShrtNm>EEX/F 20210115 F</ShrtNm><ClssfctnTp>FCECSX</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>true</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900J0JGLSFDWNFC20</Issr><TradgVnRltdAttrbts><Id>XPSF</Id><IssrReq>false</IssrReq><FrstTradDt>2018-10-30T06:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><DlvryTp>CASH</DlvryTp><AsstClssSpcfcAttrbts><Cmmdty><Pdct><Nrgy><NtrlGas><BasePdct>NRGY</BasePdct><SubPdct>NGAS</SubPdct></NtrlGas></Nrgy></Pdct><TxTp>FUTR</TxTp><FnlPricTp>PLAT</FnlPricTp></Cmmdty></AsstClssSpcfcAttrbts></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>FR</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-11-25</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XPSF</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C2WMX82</Id><FullNm>OEMB SI 20210115 PS AM C 97.00 0</FullNm><ShrtNm>Eurex/O 20210115 C AM 97.00</ShrtNm><ClssfctnTp>OCAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-11-12T06:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">97</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C2WMX90</Id><FullNm>OEMB SI 20210115 PS AM P 97.00 0</FullNm><ShrtNm>Eurex/O 20210115 P AM 97.00</ShrtNm><ClssfctnTp>OPAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-11-12T06:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>PUTO</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">97</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C2WMXA3</Id><FullNm>OEMB SI 20210115 PS AM C 98.00 0</FullNm><ShrtNm>Eurex/O 20210115 C AM 98.00</ShrtNm><ClssfctnTp>OCAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-11-12T06:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">98</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C2WMXB1</Id><FullNm>OEMB SI 20210115 PS AM P 98.00 0</FullNm><ShrtNm>Eurex/O 20210115 P AM 98.00</ShrtNm><ClssfctnTp>OPAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-11-12T06:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>PUTO</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">98</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C2WMXC9</Id><FullNm>OEMB SI 20210115 PS AM C 99.00 0</FullNm><ShrtNm>Eurex/O 20210115 C AM 99.00</ShrtNm><ClssfctnTp>OCAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-11-12T06:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">99</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C2WMXD7</Id><FullNm>OEMB SI 20210115 PS AM P 99.00 0</FullNm><ShrtNm>Eurex/O 20210115 P AM 99.00</ShrtNm><ClssfctnTp>OPAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-11-12T06:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>PUTO</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">99</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C2WMXE5</Id><FullNm>OEMB SI 20210115 PS AM C 100.00 0</FullNm><ShrtNm>Eurex/O 20210115 C AM 100.00</ShrtNm><ClssfctnTp>OCAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-11-12T06:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">100</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C2WMXF2</Id><FullNm>OEMB SI 20210115 PS AM P 100.00 0</FullNm><ShrtNm>Eurex/O 20210115 P AM 100.00</ShrtNm><ClssfctnTp>OPAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-11-12T06:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>PUTO</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">100</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C2WMXG0</Id><FullNm>OEMB SI 20210115 PS AM C 101.00 0</FullNm><ShrtNm>Eurex/O 20210115 C AM 101.00</ShrtNm><ClssfctnTp>OCAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-11-12T06:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">101</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C2WMXH8</Id><FullNm>OEMB SI 20210115 PS AM P 101.00 0</FullNm><ShrtNm>Eurex/O 20210115 P AM 101.00</ShrtNm><ClssfctnTp>OPAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-11-12T06:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>PUTO</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">101</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C2WMXJ4</Id><FullNm>OEMB SI 20210115 PS AM C 102.00 0</FullNm><ShrtNm>Eurex/O 20210115 C AM 102.00</ShrtNm><ClssfctnTp>OCAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-11-12T06:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">102</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C2WMXK2</Id><FullNm>OEMB SI 20210115 PS AM P 102.00 0</FullNm><ShrtNm>Eurex/O 20210115 P AM 102.00</ShrtNm><ClssfctnTp>OPAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-11-12T06:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>PUTO</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">102</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C2WMXL0</Id><FullNm>OEMB SI 20210115 PS AM C 103.00 0</FullNm><ShrtNm>Eurex/O 20210115 C AM 103.00</ShrtNm><ClssfctnTp>OCAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-11-12T06:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">103</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C2WMXM8</Id><FullNm>OEMB SI 20210115 PS AM P 103.00 0</FullNm><ShrtNm>Eurex/O 20210115 P AM 103.00</ShrtNm><ClssfctnTp>OPAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-11-12T06:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>PUTO</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">103</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C2WMXN6</Id><FullNm>OEMB SI 20210115 PS AM C 104.00 0</FullNm><ShrtNm>Eurex/O 20210115 C AM 104.00</ShrtNm><ClssfctnTp>OCAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-11-12T06:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">104</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C2WMXP1</Id><FullNm>OEMB SI 20210115 PS AM P 104.00 0</FullNm><ShrtNm>Eurex/O 20210115 P AM 104.00</ShrtNm><ClssfctnTp>OPAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-11-12T06:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>PUTO</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">104</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C2WMXQ9</Id><FullNm>OEMB SI 20210115 PS AM C 105.00 0</FullNm><ShrtNm>Eurex/O 20210115 C AM 105.00</ShrtNm><ClssfctnTp>OCAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-11-12T06:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">105</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C2WMXR7</Id><FullNm>OEMB SI 20210115 PS AM P 105.00 0</FullNm><ShrtNm>Eurex/O 20210115 P AM 105.00</ShrtNm><ClssfctnTp>OPAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-11-12T06:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>PUTO</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">105</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C2WMXS5</Id><FullNm>OEMB SI 20210115 PS AM C 106.00 0</FullNm><ShrtNm>Eurex/O 20210115 C AM 106.00</ShrtNm><ClssfctnTp>OCAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-11-12T06:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">106</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C2WMXT3</Id><FullNm>OEMB SI 20210115 PS AM P 106.00 0</FullNm><ShrtNm>Eurex/O 20210115 P AM 106.00</ShrtNm><ClssfctnTp>OPAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-11-12T06:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>PUTO</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">106</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C2WMXU1</Id><FullNm>OEMB SI 20210115 PS AM C 107.00 0</FullNm><ShrtNm>Eurex/O 20210115 C AM 107.00</ShrtNm><ClssfctnTp>OCAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-11-12T06:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">107</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C2WMXV9</Id><FullNm>OEMB SI 20210115 PS AM P 107.00 0</FullNm><ShrtNm>Eurex/O 20210115 P AM 107.00</ShrtNm><ClssfctnTp>OPAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-11-12T06:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>PUTO</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">107</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C2WMXW7</Id><FullNm>OEMB SI 20210115 PS AM C 108.00 0</FullNm><ShrtNm>Eurex/O 20210115 C AM 108.00</ShrtNm><ClssfctnTp>OCAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-11-12T06:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">108</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C2WMXX5</Id><FullNm>OEMB SI 20210115 PS AM P 108.00 0</FullNm><ShrtNm>Eurex/O 20210115 P AM 108.00</ShrtNm><ClssfctnTp>OPAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-11-12T06:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>PUTO</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">108</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C2WMXY3</Id><FullNm>OEMB SI 20210115 PS AM C 109.00 0</FullNm><ShrtNm>Eurex/O 20210115 C AM 109.00</ShrtNm><ClssfctnTp>OCAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-11-12T06:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">109</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C2WMXZ0</Id><FullNm>OEMB SI 20210115 PS AM P 109.00 0</FullNm><ShrtNm>Eurex/O 20210115 P AM 109.00</ShrtNm><ClssfctnTp>OPAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-11-12T06:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>PUTO</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">109</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C2WMY08</Id><FullNm>OEMB SI 20210115 PS AM C 110.00 0</FullNm><ShrtNm>Eurex/O 20210115 C AM 110.00</ShrtNm><ClssfctnTp>OCAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-11-12T06:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">110</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C2WMY16</Id><FullNm>OEMB SI 20210115 PS AM P 110.00 0</FullNm><ShrtNm>Eurex/O 20210115 P AM 110.00</ShrtNm><ClssfctnTp>OPAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-11-12T06:00:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>1</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B2NPKV68</ISIN><Nm><RefRate><Nm>ISHARES JPM USD CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>PUTO</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">110</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-12</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C2WN928</Id><FullNm>OHYU SI 20210115 PS AM C 95.00 0</FullNm><ShrtNm>Eurex/O 20210115 C AM 95.00</ShrtNm><ClssfctnTp>OCAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-11-12T06:04:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>100</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B4PY7Y77</ISIN><Nm><RefRate><Nm>ISHARES USD HY CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">95</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-13</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C2WN936</Id><FullNm>OHYU SI 20210115 PS AM P 95.00 0</FullNm><ShrtNm>Eurex/O 20210115 P AM 95.00</ShrtNm><ClssfctnTp>OPAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-11-12T06:04:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>100</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B4PY7Y77</ISIN><Nm><RefRate><Nm>ISHARES USD HY CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>PUTO</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">95</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-13</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C2WN944</Id><FullNm>OHYU SI 20210115 PS AM C 96.00 0</FullNm><ShrtNm>Eurex/O 20210115 C AM 96.00</ShrtNm><ClssfctnTp>OCAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-11-12T06:04:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>100</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B4PY7Y77</ISIN><Nm><RefRate><Nm>ISHARES USD HY CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">96</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-13</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>DE000C2WN951</Id><FullNm>OHYU SI 20210115 PS AM P 96.00 0</FullNm><ShrtNm>Eurex/O 20210115 P AM 96.00</ShrtNm><ClssfctnTp>OPAIPS</ClssfctnTp><NtnlCcy>USD</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>529900UT4DG0LG5R9O07</Issr><TradgVnRltdAttrbts><Id>XEUR</Id><IssrReq>false</IssrReq><FrstTradDt>2018-11-12T06:04:00Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>100</PricMltplr><UndrlygInstrm><Sngl><Indx><ISIN>IE00B4PY7Y77</ISIN><Nm><RefRate><Nm>ISHARES USD HY CB ETF</Nm></RefRate></Nm></Indx></Sngl></UndrlygInstrm><OptnTp>PUTO</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">96</Amt></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>PHYS</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-13</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XEUR</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>CH0483993256</Id><FullNm>UBS MSF CAL 21C</FullNm><ShrtNm>UBS London/C Wt MSFT 156 20210115</ShrtNm><ClssfctnTp>RWSNCA</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>BFM8T61CT2L1QCEMIK50</Issr><TradgVnRltdAttrbts><Id>UBSI</Id><IssrReq>false</IssrReq><FrstTradDt>2019-09-19T14:09:01.67Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>0.1</PricMltplr><UndrlygInstrm><Sngl><ISIN>US5949181045</ISIN></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">156</Amt><Sgn>true</Sgn></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>CASH</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>UBSI</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>CH0483995053</Id><FullNm>UBS NVD CAL 21C</FullNm><ShrtNm>UBS London/C Wt NVDA 180 20210115</ShrtNm><ClssfctnTp>RWSNCA</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>BFM8T61CT2L1QCEMIK50</Issr><TradgVnRltdAttrbts><Id>UBSI</Id><IssrReq>false</IssrReq><FrstTradDt>2019-08-22T16:21:30.12Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>0.1</PricMltplr><UndrlygInstrm><Sngl><ISIN>US67066G1040</ISIN></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">180</Amt><Sgn>true</Sgn></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>CASH</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>UBSI</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>CH0483995558</Id><FullNm>UBS NVD CAL 21C</FullNm><ShrtNm>UBS London/C Wt NVDA 200 20210115</ShrtNm><ClssfctnTp>RWSNCA</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>BFM8T61CT2L1QCEMIK50</Issr><TradgVnRltdAttrbts><Id>UBSI</Id><IssrReq>false</IssrReq><FrstTradDt>2019-09-16T18:40:19.04Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>0.1</PricMltplr><UndrlygInstrm><Sngl><ISIN>US67066G1040</ISIN></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">200</Amt><Sgn>true</Sgn></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>CASH</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-12-17</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>UBSI</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>CH0484021032</Id><FullNm>UBS PRG CAL 21C</FullNm><ShrtNm>UBS LONDON/C WT PG 115 20210115</ShrtNm><ClssfctnTp>RWSNCA</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>BFM8T61CT2L1QCEMIK50</Issr><TradgVnRltdAttrbts><Id>UBSI</Id><IssrReq>false</IssrReq><FrstTradDt>2019-06-18T19:15:38Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>0.1</PricMltplr><UndrlygInstrm><Sngl><ISIN>US7427181091</ISIN></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">115</Amt><Sgn>true</Sgn></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>CASH</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>UBSI</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>CH0484021040</Id><FullNm>UBS PRG CAL 21C</FullNm><ShrtNm>UBS London/C Wt PG 120 20210115</ShrtNm><ClssfctnTp>RWSNCA</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>BFM8T61CT2L1QCEMIK50</Issr><TradgVnRltdAttrbts><Id>UBSI</Id><IssrReq>false</IssrReq><FrstTradDt>2019-06-21T17:30:01Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>0.1</PricMltplr><UndrlygInstrm><Sngl><ISIN>US7427181091</ISIN></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">120</Amt><Sgn>true</Sgn></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>CASH</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>UBSI</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>CH0484021057</Id><FullNm>UBS PRG CAL 21C</FullNm><ShrtNm>UBS London/C Wt PG 125 20210115</ShrtNm><ClssfctnTp>RWSNCA</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>BFM8T61CT2L1QCEMIK50</Issr><TradgVnRltdAttrbts><Id>UBSI</Id><IssrReq>false</IssrReq><FrstTradDt>2019-09-04T12:25:30Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>0.1</PricMltplr><UndrlygInstrm><Sngl><ISIN>US7427181091</ISIN></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">125</Amt><Sgn>true</Sgn></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>CASH</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>UBSI</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>CH0486072439</Id><FullNm>UBS AMZ CAL 21C</FullNm><ShrtNm>UBS London/C Wt AMZN 2680 20210115</ShrtNm><ClssfctnTp>RWSNCA</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>BFM8T61CT2L1QCEMIK50</Issr><TradgVnRltdAttrbts><Id>UBSI</Id><IssrReq>false</IssrReq><FrstTradDt>2019-07-15T14:21:13.19Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>0.01</PricMltplr><UndrlygInstrm><Sngl><ISIN>US0231351067</ISIN></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">2680</Amt><Sgn>true</Sgn></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>CASH</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>UBSI</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>CH0501364886</Id><FullNm>EMT SGIS CallableReverseConvertible</FullNm><ShrtNm>SG Issuer/6.708 BRC 20210115 Mlt KI</ShrtNm><ClssfctnTp>DEDFSB</ClssfctnTp><NtnlCcy>CHF</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>549300QNMDBVTHX8H127</Issr><TradgVnRltdAttrbts><Id>XSGA</Id><IssrReq>false</IssrReq><AdmssnApprvlDtByIssr>2019-10-14T01:00:00Z</AdmssnApprvlDtByIssr><ReqForAdmssnDt>2019-10-14T01:00:00Z</ReqForAdmssnDt><FrstTradDt>2019-10-14T01:00:00Z</FrstTradDt><TermntnDt>2021-01-15T01:00:00Z</TermntnDt></TradgVnRltdAttrbts><DebtInstrmAttrbts><TtlIssdNmnlAmt Ccy="CHF">156000</TtlIssdNmnlAmt><MtrtyDt>2021-01-15</MtrtyDt><NmnlValPerUnit Ccy="CHF">156000</NmnlValPerUnit><IntrstRate><Fxd>0</Fxd></IntrstRate></DebtInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>FR</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-11-20</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>XSGA</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>CH0512281723</Id><FullNm>Call Optionsschein auf Electronic Arts</FullNm><ShrtNm>UBS LONDON/C WT EA 105 20210115</ShrtNm><ClssfctnTp>RWSNCA</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>BFM8T61CT2L1QCEMIK50</Issr><TradgVnRltdAttrbts><Id>STUB</Id><IssrReq>true</IssrReq><AdmssnApprvlDtByIssr>2019-11-28T10:44:57Z</AdmssnApprvlDtByIssr><FrstTradDt>2019-11-29T08:00:00Z</FrstTradDt><TermntnDt>2021-01-15T22:00:00Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>0.1</PricMltplr><UndrlygInstrm><Sngl><ISIN>US2855121099</ISIN></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">105</Amt><Sgn>true</Sgn></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>CASH</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>STUB</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>CH0512288801</Id><FullNm>UBS IDX CAL 21C</FullNm><ShrtNm>UBS LONDON/C WT BKNG 1850 20210115</ShrtNm><ClssfctnTp>RWSNCA</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>BFM8T61CT2L1QCEMIK50</Issr><TradgVnRltdAttrbts><Id>UBSI</Id><IssrReq>false</IssrReq><FrstTradDt>2020-01-22T14:43:27Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>0.01</PricMltplr><UndrlygInstrm><Sngl><ISIN>US09857L1089</ISIN></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">1850</Amt><Sgn>true</Sgn></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>CASH</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>UBSI</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>CH0512292936</Id><FullNm>UBS IDX CAL 21C</FullNm><ShrtNm>UBS LONDON/C WT BKNG 2100 20210115</ShrtNm><ClssfctnTp>RWSNCA</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>BFM8T61CT2L1QCEMIK50</Issr><TradgVnRltdAttrbts><Id>UBSI</Id><IssrReq>false</IssrReq><FrstTradDt>2020-01-30T08:21:32Z</FrstTradDt><TermntnDt>2021-01-15T23:59:59Z</TermntnDt></TradgVnRltdAttrbts><DerivInstrmAttrbts><XpryDt>2021-01-15</XpryDt><PricMltplr>0.01</PricMltplr><UndrlygInstrm><Sngl><ISIN>US09857L1089</ISIN></Sngl></UndrlygInstrm><OptnTp>CALL</OptnTp><StrkPric><Pric><MntryVal><Amt Ccy="USD">2100</Amt><Sgn>true</Sgn></MntryVal></Pric></StrkPric><OptnExrcStyle>AMER</OptnExrcStyle><DlvryTp>CASH</DlvryTp></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>DE</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2021-01-16</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>UBSI</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>CH0513597234</Id><FullNm>Leonteq Securities AG EO-Anl. 2020(21) Saint-Gobain</FullNm><ShrtNm>LEONTEQ SECUR/5.53 RVCV 20210120</ShrtNm><ClssfctnTp>DECFTS</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>ML61HP3A4MKTTA1ZB671</Issr><TradgVnRltdAttrbts><Id>FRAB</Id><IssrReq>true</IssrReq><AdmssnApprvlDtByIssr>2020-01-30T23:00:00Z</AdmssnApprvlDtByIssr><FrstTradDt>2020-02-03T07:00:00Z</FrstTradDt><TermntnDt>2021-01-15T22:59:59Z</TermntnDt></TradgVnRltdAttrbts><DebtInstrmAttrbts><TtlIssdNmnlAmt Ccy="EUR">10000000</TtlIssdNmnlAmt><MtrtyDt>2021-01-20</MtrtyDt><NmnlValPerUnit Ccy="EUR">1000</NmnlValPerUnit><IntrstRate><Fxd>5.53</Fxd></IntrstRate></DebtInstrmAttrbts><DerivInstrmAttrbts><PricMltplr>33.156</PricMltplr><UndrlygInstrm><Sngl><ISIN>FR0000125007</ISIN></Sngl></UndrlygInstrm></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>FR</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-07-19</FrDt><ToDt>2021-01-16</ToDt></FrDtToDt></PblctnPrd><RlvntTradgVn>FRAB</RlvntTradgVn></TechAttrbts></TermntdRcrd></FinInstrm><FinInstrm><TermntdRcrd><FinInstrmGnlAttrbts><Id>CH0513597242</Id><FullNm>Leonteq Securities AG EO-Anl. 2020(21) Vivendi</FullNm><ShrtNm>LEONTEQ SECUR/3.44 RVCV 20210120</ShrtNm><ClssfctnTp>DECFTS</ClssfctnTp><NtnlCcy>EUR</NtnlCcy><CmmdtyDerivInd>false</CmmdtyDerivInd></FinInstrmGnlAttrbts><Issr>ML61HP3A4MKTTA1ZB671</Issr><TradgVnRltdAttrbts><Id>FRAB</Id><IssrReq>true</IssrReq><AdmssnApprvlDtByIssr>2020-01-30T23:00:00Z</AdmssnApprvlDtByIssr><FrstTradDt>2020-02-03T07:00:00Z</FrstTradDt><TermntnDt>2021-01-15T22:59:59Z</TermntnDt></TradgVnRltdAttrbts><DebtInstrmAttrbts><TtlIssdNmnlAmt Ccy="EUR">10000000</TtlIssdNmnlAmt><MtrtyDt>2021-01-20</MtrtyDt><NmnlValPerUnit Ccy="EUR">1000</NmnlValPerUnit><IntrstRate><Fxd>3.44</Fxd></IntrstRate></DebtInstrmAttrbts><DerivInstrmAttrbts><PricMltplr>47.303</PricMltplr><UndrlygInstrm><Sngl><ISIN>FR0000127771</ISIN></Sngl></UndrlygInstrm></DerivInstrmAttrbts><TechAttrbts><RlvntCmptntAuthrty>FR</RlvntCmptntAuthrty><PblctnPrd><FrDtToDt><FrDt>2020-07-19</FrDt><ToDt>2'''

In [ ]:
root = et.fromstring(my_string)

# Print the root element and its children
print(root.tag)
for child in root:
    print(child.tag, child.attrib)

In [ ]:
with open('data2.xml', 'wb') as f:
    f.write(con)
mytree = et.parse('data2.xml')

In [46]:


# Parse the XML file
tree = ET.parse('data.xml')
root = tree.getroot()

# Open a CSV file for writing
with open('parvesh.csv', mode='w', newline='') as csv_file:
    writer = csv.writer(csv_file)

    # Write the header row
    writer.writerow(['id', 'file_name', 'download_link', 'publication_date'])

    # Write data rows
    for doc in root.findall("./result[@name='response']/doc"):
        id = doc.find("./str[@name='id']").text
        file_name = doc.find("./str[@name='file_name']").text
        download_link = doc.find("./str[@name='download_link']").text
        publication_date = doc.find("./date[@name='publication_date']").text

        writer.writerow([id, file_name, download_link, publication_date])

with open('Parvesh.csv', newline='')as csvfile:
    reader= csv.reader(csvfile)
    for row in reader:
        print(','.join(row))

id,file_name,download_link,publication_date
46015,DLTINS_20210117_01of01.zip,http://firds.esma.europa.eu/firds/DLTINS_20210117_01of01.zip,2021-01-17T00:00:00Z
46051,DLTINS_20210119_01of02.zip,http://firds.esma.europa.eu/firds/DLTINS_20210119_01of02.zip,2021-01-19T00:00:00Z
46052,DLTINS_20210119_02of02.zip,http://firds.esma.europa.eu/firds/DLTINS_20210119_02of02.zip,2021-01-19T00:00:00Z
46032,DLTINS_20210118_01of01.zip,http://firds.esma.europa.eu/firds/DLTINS_20210118_01of01.zip,2021-01-18T00:00:00Z


In [ ]:
 # Write the header row
    writer.writerow(['id', 'file_name', 'download_link', 'publication_date'])

    # Write data rows
    for doc in root.findall("./result[@name='response']/doc"):
        id = doc.find("./str[@name='id']").text
        file_name = doc.find("./str[@name='file_name']").text
        download_link = doc.find("./str[@name='download_link']").text
        publication_date = doc.find("./date[@name='publication_date']").text

        writer.writerow([id, file_name, download_link, publication_date])

with open('Parvesh.csv', newline='')as csvfile:
    reader= csv.reader(csvfile)
    for row in reader:
        print(','.join(row))

In [ ]:
with open('final.csv', mode='w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    
    writer.writerow(['id', 'FullNm', 'ClssfctnTp', 'CmmdtyDerivInd', "NtnlCcy"])
    for i in root.findall('.//doc'):
        
        id = doc.find('str[@name="Id"]').text
        FullNm = FullNm = doc.find('str[@name="FullNm"]').text
        doc.find('str[@name="ClssfctnTp"]').text
        CmmdtyDerivInd = doc.find('str[@name="CmmdtyDerivInd"]').text
        NtnlCcy = doc.find('str[@name="NtnlCcy"]').text
        
        write.writerow([id, FullNm, ClssfctnTp, CmmdtyDerivInd, NtnlCcy])
        
        
with open('final.csv', newline='')as csvfile:
    reader= csv.reader(csvfile)
    for row in reader:
        print(','.join(row))

In [51]:
import pandas as pd
df = pd.read_xml(r'D:\New folder\Coding\Pyhton Projects\SteelEye\DLTINS_20210117_01of01.xml')

ImportError: lxml not found, please install or use the etree parser.